![Steam_Logo](Steam_Logo.jpg)

**Steam** es una plataforma de distribución digital de videojuegos desarrollada por Valve Corporation. Fue lanzada en septiembre de 2003 como una forma para Valve de proporcionar actualizaciones automáticas a sus juegos, pero finalmente se expandió para incluir juegos de terceros.

# Paso 1: Comenzando nuestro proceso ETL
Comenzamos leyendo el Glosario de Datos para comprender la estructura y el contenido de los tres archivos proporcionados en la documentación del proyecto:

1. **steam_games.json.gz**: Este conjunto de datos contiene información completa sobre los juegos disponibles en Steam. Incluye detalles como títulos de juegos, géneros, fechas de lanzamiento, etc., lo que nos permite obtener información sobre el **catálogo de juegos**.

2. **user_reviews.json**: El conjunto de datos de reseñas de usuarios proporciona comentarios generados por los usuarios para los juegos de Steam. Contiene datos como el texto de la reseña y los votos de retroalimentación de calificación, lo que nos permite analizar los sentimientos y experiencias de los usuarios de manera efectiva.

3. **users_items.json**: En este conjunto de datos, encontramos información relacionada con los usuarios de Steam y sus colecciones de objetos dentro del juego. Los datos interesantes de este conjunto incluyen las preferencias de los usuarios y los objetos específicos que poseen.

Una vez que revisamos la estructura, las definiciones de columnas y la configuración, procedimos a cargarlos en Dataframes de Pandas para acceder a la información y transformarla y procesarla.<br>
<br>

## 1.1 Juegos

Procedimos con el [ETL_juegos](ETL_juegos.ipynb): **steam_games.json.gz** sin inconvenientes:<br>

El dataframe se llamó df_juegos y contiene:<br>
* 120.445 filas
* 13 columnas
<br>
Al usar Data Wrangler, obtenemos una idea general del conjunto de datos. Contiene muchos valores faltantes; la mayoría de las columnas tienen hasta un 70% de valores faltantes en el conjunto de datos.<br>

> Eliminaremos la información faltante, ya que, en primer lugar, no podemos completar la información faltante debido a su diversidad y origen desconocido.<br>
> Además, eliminar la información faltante aún nos proporciona suficiente información para nuestro proyecto. No hay pérdida.

Ahora nos quedan 22.530 filas y 13 columnas de información.<br>
Ahora debemos evaluar cada columna para decidir cuáles son importantes para el proyecto y cuáles se pueden eliminar.<br>
<br>
Para continuar, eliminaremos los duplicados: solo se eliminaron 2 registros.

#### Ahora podemos comenzar a definir qué columnas debemos conservar. <br>
Las columnas que ya sabemos que son necesarias para las funciones de la API son: <br>

*   **developr** para la función **def desarrollador**
*   **genre** para la función **def usuario_por_género**
*   **release_date** para la función **def mejor_desarrollador_año**
*   **price** para la función **def datos_usuario**
*   **title** para **def mejor_desarrollador_año**
*   **id** identifica cada juego por su valor único de id
*   **publisher** para la función **def desarrollador**

##### Estas columnas anteriores se transformarán y manipularán como parte del modelo.


#### También podemos comenzar a evaluar la información de otras columnas para eliminar las que no necesitamos de nuestro conjunto de datos.<br>
##### Estas son:
*   **app_name** información similar a la del título
*   **url** no aporta valor al proyecto previsto
*   **tags** similar a los géneros y no aporta valor al proyecto previsto
*   **discount** no aporta valor al proyecto previsto
*   **reviews_url** no aporta valor al proyecto previsto
*   **specs** no aporta valor al proyecto previsto
*   **early_access** no aporta valor al proyecto previsto

##### Por lo tanto, todas las columnas anteriores se eliminarán.

Añadiendo valor a nuestros datos:
*   Convertimos **price** en una columna completamente de tipo float, de modo que toda la información sobre gratuitos y promociones se convierte en ceros.
*   Creamos una nueva columna = **release_year** y eliminamos la columna anterior **release_date**. Esta nueva columna se utilizará en la función **top_developer**.
*   Eliminamos todas las columnas que no aportan al modelo o las funciones.

#### Ahora queda una última columna que debemos analizar y transformar: 'géneros'
##### géneros
Esta columna contiene una lista en cada una de las filas que indica a qué géneros pertenece el juego. <br>
Un **genre** es una categoría que agrupa juegos según sus objetivos, historia y jugabilidad.<br>
Ayuda a los usuarios y desarrolladores a identificar juegos con estilos y temas similares, como los desafíos intensos en los juegos de acción o el desarrollo de personajes y las misiones en los juegos de rol.<br>
En esta columna, para cada juego (fila), encontramos una lista de géneros que presenta el juego. Esta información se utilizará en la función **user_for_genre** y en nuestro modelo de recomendación. Sin embargo, abordaremos esto en nuestro análisis exploratorio de datos (EDA); por ahora, continuaremos con nuestras etapas de ETL.

Hemos almacenado el dataframe transformado en [games.parquet](games.parquet) en la carpeta "data".

## 1.2 Reseñas de Usuarios

Al pasar al segundo archivo [ETL_user_reviews](ETL_user_reviews.ipynb), nos enfrentamos a nuestro primer desafío:<br>
El procedimiento estándar para leer el archivo utilizado en el primer archivo resultó en errores tras errores, porque los datos dentro del archivo .json no estaban en el formato esperado.<br>
Después de investigar, aprendí que cuando el archivo JSON que intentas cargar no está correctamente formateado (como es el caso aquí), puedes evaluarlo línea por línea utilizando **ast.literal_eval()**. <br>

Después de buscar extensamente y profundizar en Stack Overflow, encontré un artículo que fue un avance:

[Convert JSON to pd.DataFrame](https://stackoverflow.com/questions/55338899/convert-json-to-pd-dataframe/65427497#65427497)<br>
[JSONDecodeError](https://bobbyhadz.com/blog/python-jsondecodeerror-expecting-property-name-enclosed-in-double-quotes)<br>

El dataframe **df_reviews** (**user_reviews.json.gz**) contiene:<br>
*   25.485 filas
*   3 columnas
<br>
Primero procedemos a identificar valores faltantes o duplicados.
Encontramos 623 registros duplicados, validamos que realmente estén duplicados y finalmente eliminamos el valor duplicado, conservando solo la primera ocurrencia del valor.<br>
Luego avanzamos a la columna de reseñas, que es de gran interés para nuestro proyecto.<br>

Fue fácil detectar que la tercera columna es una colección de listas de diccionarios, y que los datos dentro de la última columna son vitales para el desarrollo del análisis de sentimientos.

```
df_reviews.iloc[100,2]
```
[{'funny': '',
  'posted': 'Posted October 13, 2014.',
  'last_edited': '',
  'item_id': '209870',
  'helpful': '3 of 8 people (38%) found this review helpful',
  'recommend': True,
  'review': 'Its a very fun game i recomend as its nearly like TITANFALL but its FREE!Play this game now'}]

#### Necesitamos extraer en columnas la información presente dentro de cada campo de la tercera columna. <br>
Los campos que se convertirán en columnas son: graciosos, publicados, editados por última vez, id del ítem, útil, recomendado y reseña.
Después de varios intentos con diferentes enfoques, encontré una página con una explicación de un ejercicio similar que adapté para resolver la lista anidada de diccionarios en columnas y combinaciones de registros. <br>

[Convert list of nested dictionary into Pandas dataframe](https://www.geeksforgeeks.org/python-convert-list-of-nested-dictionary-into-pandas-dataframe/)

Finalmente, el dataframe desanidado contiene:

* 59.305 filas
* 9 columnas ([‘user_id’, ‘user_url’, ‘funny’, ‘posted’, ‘last_edited’, ‘item_id’, ‘helpful’, ‘recommend’, ‘review_text’]) Por lo tanto, el resultado final es un dataframe con 25.485 filas, sin valores faltantes y las siguientes columnas:<br>
* **user_id**: identificador único para cada usuario en la plataforma.<br>
* **user_url**: URL que aloja el perfil del usuario en Steam Community.<br>
* **reviews**: contiene una lista de diccionarios.Para cada usuario, enumera todas las reseñas publicadas por el usuario:
    * **funny**: indica si otro usuario califica como graciosa la reseña del usuario
    * **posted**: fecha en que se realizó la publicación, formato: “Publicado el 21 de abril de 2011”.
    * **last_edited**: fecha en que se editó la publicación.  
    * **item_id**:es el ID del juego, único para cada juego.
    * **helpful**: otros usuarios califican si la reseña fue útil
    * **recommend**: un valor booleano que indica si el usuario recomienda los juegos o no
    * **review**: es la reseña escrita por el usuario que se publica en la URL del juego.

De todas estas columnas, encontramos dos con valores faltantes en su mayoría, y los valores faltantes en el formato ‘’ se cambiaron a None. <br> Después de revisar la cantidad de valores None en todas las columnas agregadas, decidimos eliminar las columnas:<br>

- **funny** con más del 86% de valores None.
- **last_edited** con más del 89% de valores None.

Luego procedimos a transformar la columna ‘publicado’ de un formato ‘Publicado {mes} {día}, {año}’ en una nueva columna ‘fecha’. En esta columna encontramos 9929 registros con el año faltante, que se completaron con el año actual (2024) para diferenciarlos del resto de los valores extraídos correctamente. Estos registros de 2024 no se considerarán para la función mejor_desarrollador_año, pero la información restante aún está disponible para el modelo.<br>

El dataframe resultante [reviews.parquet](data\reviews.parquet) resume 58.400 filas y 9 columnas.<br>

## 1.2 User_Items
Finalmente, avanzamos con nuestro tercer y último archivo [ETL_reviews](01.%20ETL/ETL_reviews.ipynb): **users_items.json.gz**
Obtenemos el mismo error que en el segundo archivo, por lo que debemos cargar la información utilizando la función literal de ast.<br>

Una vez más, encontramos otra columna, items, con una lista anidada de diccionarios, y la información en esta columna es la lista de elementos (juegos) que cada usuario compró. Esta es toda la información valiosa que necesitamos para desarrollar nuestras funciones para nuestro modelo.<br> 
Antes de desanidar la columna ‘items’, verificamos una vez más los duplicados utilizando el campo ‘user_id’.<br> 
Encontramos 1357 registros duplicados, que después de validar que estaban duplicados, procedimos a eliminarlos y mantener solo la primera ocurrencia.<br>

Finalmente, procedemos como con el archivo de reseñas anterior, aprendiendo cuál es la estructura de los datos que necesitamos extraer de la columna ‘items’.<br>

Las columnas que agregaremos a nuestro df_items son:

```
df_items.iloc[10,4]
```
[{'item_id': '4000',
  'item_name': "Garry's Mod",
  'playtime_forever': 2644,
  'playtime_2weeks': 0},
 {'item_id': '1250',
  'item_name': 'Killing Floor',
  'playtime_forever': 30266,
  'playtime_2weeks': 0},
 {'item_id': '35420',
  'item_name': 'Killing Floor Mod: Defence Alliance 2',
  'playtime_forever': 54,
  'playtime_2weeks': 0},

...
  'playtime_2weeks': 462},
 {'item_id': '444640',
  'item_name': 'Bloons TD Battles',
  'playtime_forever': 809,
  'playtime_2weeks': 0}]

  Entonces necesitamos iterar a través del campo de elementos para cada fila, con el fin de extraer para cada usuario la siguiente información:<br>
  - **'item_id'**: '444640' --> es el identificador único del game **game_id**<br>
  - **'item_name'**: 'Bloons TD Battles',--> **item_name** nombre del game<br>
  - **'playtime_forever'**: 809,--> La cantidad total de tiempo en minutos que el usuario jugó el juego en total **minutes**<br>
  - **'playtime_2weeks'**: 0}] --> La cantidad total de tiempo en minutos que el usuario jugó el juego en **las últimas dos semanas**<br>

Entonces ejecutamos nuestro código para convertir esta información en columnas para cada usuario.<br> 
<br> 
Luego obtenemos un dataframe de:<br>

* 5.153.209 filas<br>
*8 columnas<br> <br> 
Ahora procedemos a obtener más información sobre las diferentes columnas y los datos para decidir qué columna es relevante para nuestro proyecto y cuál se puede eliminar.<br>
###  Información valiosa y datos desechables <br>
Ahora que tenemos una idea de la estructura del dataframe y la información, podemos concluir que hay muchas columnas que podemos eliminar para mantener nuestros datos manejables y lo más compactos posible.<br>

**Columnas a eliminar:**<br>
-   **items_count** es una columna que, dada la información, se puede calcular a demanda.<br>
-   **steam_id** contiene la misma información que  **user_id**
-   **user_url** es el url del usuario, y no lo utilizaremos en particular en nuestro análisis<br>
<br>

El dataframe resultante [items.parquet](data\items.parquet) resume 5.094.082 filas y 5 columnas.


# Paso 2: Resumen del Análisis EDA
Ahora que tenemos los tres archivos limpios y listos, podemos comenzar a obtener información sobre los datos que se utilizarán en nuestro proyecto de modelo de recomendación.<br>
Aquí encontrarás un resumen del trabajo realizado en el análisis exploratorio de datos (EDA). <br> 
Para obtener información más detallada y visualizaciones:<br>

[EDA_games](0.%20EDA/EDA_games.ipynb)<br>

[EDA_reviews](02.%20EDA/EDA_reviews.ipynb)<br>

[EDA_items](02.%20EDA/EDA_items.ipynb)<br>

## 2.1 Juegos
En el Análisis EDA, nos centraremos principalmente en:<br>

- buscar valores atípicos
- estadísticas y descripción de los datos
- transformación de datos
- Variables:
    - **item_id**: esta columna contiene el identificador único del juego.
    - **item_name**: este es el nombre del juego.
    - **developer**: es un desarrollador de software especializado en el desarrollo de videojuegos, el proceso y las disciplinas relacionadas con la creación de videojuegos.
    - **publisher**: una editorial de videojuegos es una empresa que publica videojuegos que han sido desarrollados internamente por la editorial o externamente por un desarrollador de videojuegos.
    - **release_year**: es el año en que se lanzó el juego.
    - **genres**: esta columna contiene una lista de géneros para cada juego, que transformaremos con diferentes enfoques.<br>

### Un resumen de lo que encontrarás en el cuaderno:
#### Precio
*   Descubrir si nuestra información de precios tiene valores atípicos.
*   Juegos más caros.
*   Precio promedio de los juegos.
*   Rango de precios.
*   Distribución de precios.
*   Estadísticas de precios.

#### Release Year
*   Rango de años.
*   Juegos lanzados por año.
*   Los cinco años con más juegos lanzados.

#### Publishers
*   Las diez editoriales con la mayor cantidad de juegos publicados.

#### Developers
*   Los diez desarrolladores con la mayor cantidad de juegos publicados.
*   Los desarrolladores menos populares o relevantes, con la menor cantidad de juegos publicados.
*   Las editoriales menos importantes, con la menor cantidad de juegos publicados.

#### Genres
*   Los géneros más populares, con la mayor cantidad de juegos publicados.
*   Los géneros menos populares, con la menor cantidad de juegos publicados.

## Análisis de correlación entre géneros y precios
### ¿Tienen los géneros un impacto en el precio de los juegos?

## 2.2 Reviews

**El propósito final de este análisis es definir, para el modelo de recomendación solicitado y las diferentes funciones, para cada conjunto de datos y función, las columnas y datos correspondientes que serán necesarios**.

### Durante el proceso ETL, ya nos ocupamos de:
[ETL_user_reviews](01.%ETL\ETL_user_reviews.ipynb)<br>

- valores faltantes y limpieza de datos
- eliminación de columnas de información irrelevante
- eliminación de filas con información inexistente o no válida.
- eliminación de valores duplicados
- corrección de todos los tipos de datos para cada dato y columna

### Análisis de sentimientos
El conjunto de datos que utilizaremos ya ha pasado por la transformación de análisis de sentimientos, por lo que, en lugar de trabajar en el análisis exploratorio de datos (EDA) con las reseñas, tendremos una columna llamada ‘sentimiento’ que muestra el resultado del proceso de análisis de sentimientos.<br>

### Por lo tanto, en el Análisis EDA, nos centraremos principalmente en:
- buscar valores atípicos
- estadísticas y descripción de los datos
- transformación de datos
- Variables:
- user_id: esta columna contiene el identificador único para cada uno de los usuarios en Steam.
- user_url: esta columna contiene la URL al perfil de cada usuario en Steam.
- item_id: esta columna contiene el identificador único para el juego.
- helpful: en esta columna, otros usuarios califican si la reseña fue útil.
- recommend: esta columna es un valor booleano que indica si el usuario - recomienda los juegos o no.
- date: es la fecha en que se publicó la reseña. A partir de esta columna, extraemos la columna year.
- year: es el año en que se lanzó el juego.
- sentiment: esta columna contiene el valor de clasificación del análisis de sentimientos:<br>
0 --> Negativo <br> 
1 --> Neutral <br> 
2  --> Positivo <br>

## Un resumen de lo que encontrarás en el notebook:
#### user_id
- Número total de usuarios en la plataforma.
- Cálculo del número promedio de reseñas por usuario.
- Exploración de la distribución del número de reseñas por usuario.
- Determinación del usuario con el mayor número de reseñas.
- Identificación del usuario con el mayor número promedio de reseñas por año.

**Hallazgos:**

* Solo el 30% de los usuarios en la plataforma posteó reseñas, esto es 25.446 unique user_id
* Este 30% de los usuarios en promedio publica 2.3 reseñas:
* De este 30%, el 53% de los usuarios publica solo una reseña
* Y menos del 2% de los usuarios que publican, publican más de 8 reseñas

#### item_id
- Cuántos juegos tienen reseñas
- Qué porcentaje de juegos tiene reseñas del total de juegos disponibles en la plataforma
- Exploración de la distribución del número de reseñas por juego
- Promedio de reseñas por juego
- Identificación de los 10 juegos con mayor número de reseñas
- Juegos con más de 500 reseñas

**Conclusiones:**

* El 16% de todos los juegos registrados en la plataforma reciben reseñas, 3,682 de un total de 22,530.
* El 15% de los juegos reciben entre 1 y 12 reseñas
* El promedio de reseñas por juego es de 15 reseñas
* El 67% de todas las reseñas se concentran en solo 12 juegos.

### Helpful
En esta columna, otros usuarios califican si la reseña fue útil.
Así es como muchos usuarios han calificado la reseña publicada como útil.
Vamos a transformar esta columna para obtener una idea de cómo otros usuarios califican la reseña del usuario.

**Conclusiones:**
- La mayoría de las reseñas no son calificadas por otros usuarios, el 61% de ellas no son calificadas.
- El 17% de las reseñas calificadas son consideradas útiles por otros usuarios.

## Análisis de Recomendación y Sentimiento
**Realizamos un análisis bivariado**<br>

En el caso de estas dos variables, vamos a realizar el EDA como un análisis bivariable.
Esta columna es un valor booleano que indica si el usuario recomienda los juegos o no. Esto es, si la recomendación es positiva, hay un 1 y si no, un cero.
Podemos usar esta columna para evaluar o validar nuestro resultado del análisis de sentimiento.
- Cuántos usuarios publican una recomendación positiva
- Cuántos usuarios publican una recomendación positiva y una reseña de sentimiento positivo
- Cuántas recomendaciones positivas coinciden con un sentimiento positivo
- Cuántos sentimientos negativos coinciden con una recomendación negativa

#### Análisis de Sentimiento y Recomendación
Vamos a considerar que todos los sentimientos neutros son positivos, y vamos a realizar un tipo de puntuación de precisión al análisis de sentimiento utilizando las bibliotecas NLTK y TextBlob dada la columna de recomendación.
Esto es para analizar de alguna manera qué tan precisa es nuestra análisis de sentimiento.

**Conclusiones**
A partir de la evaluación de precisión entre ambos análisis de sentimiento, concluimos que el análisis NLTK es el que tiene mejor rendimiento.
Ahora podemos revisar los resultados en comparación con la recomendación del usuario:

* **Negativos** Podemos ver que para la recomendación Verdadera, es decir, que el usuario recomienda el juego incluso si la reseña es negativa, el porcentaje de recomendación es de aproximadamente el 70%, lo que significa que la mayoría de los usuarios, aunque consideren que el juego no es bueno, todavía lo recomiendan.
* **Neutral** En este caso, la reseña siendo neutral no es una reseña negativa, por lo que no es sorprendente que los usuarios aún recomienden el juego incluso si la reseña no es mala. Y además, nuestro análisis de sentimiento funciona muy bien reconociendo reseñas positivas pero no tan bien clasificando reseñas neutrales.
* **Positivos** En este caso, confirmamos lo que nuestro test de precisión mostró, nuestro análisis de sentimiento es bueno clasificando reseñas positivas, y las reseñas positivas coinciden en el 91% de los casos.
Dado que nuestro análisis fue un enfoque rápido y simple con muchas oportunidades, los resultados son buenos, considerando todo.

## 2.3 Items
### Durante el proceso ETL ya nos ocupamos de:

[ETL_items](01.%20ETL/ETL_items.ipynb)<br>

-   valores faltantes y limpieza de datos
-   eliminación de columnas de información irrelevante
-   eliminación de filas con información no existente o no válida
-   eliminación de valores duplicados
-   corrección de todos los tipos de datos para cada columna y dato

### Entonces en el **Análisis EDA** nos enfocaremos principalmente en:
-   búsqueda de valores atípicos
-   estadísticas y descripción de datos
-   transformación de datos
-   Variables:
    - user_id: Este user_id es el identificador único para cada uno de los usuarios en la columna de la plataforma
    - item_id: identificador único del juego en la plataforma
    - item_name: nombre del juego o aplicación
    - playtime_forever: El número total de minutos jugados "registrados"
    - playtime_2weeks: El número total de minutos jugados en las últimas 2 semanas
-   Análisis multivariable:

### Un resumen de lo que encontrarás en el cuaderno:

#### User_id
Este user_id es el identificador único para cada uno de los usuarios en la plataforma.
**Hallazgos**
- Cuántos usuarios están registrados en la plataforma

#### item_id
- Cuántos juegos son elementos en las cuentas de los usuarios
- Porcentaje del total de juegos
- Promedio de cantidad de juegos por usuario
- Los 10 elementos más vendidos (más usuarios lo tienen)
- Los 10 juegos más jugados en las últimas 2 semanas

#### Atípicos para item_id por usuario

**Hallazgos**
* La cantidad promedio de juegos por usuario es de 71 juegos
* Más del 50% de los usuarios poseen más de 40 juegos cada uno.
* Solo hay 39 usuarios con más de 2000 juegos cada uno, esto se considerará como atípicos pero representan menos del 0.056%
* La **cantidad máxima** de juegos propiedad de un usuario es de *7762 juegos*
- Los 10 elementos más vendidos (más usuarios lo tienen)

#### item_name
Este es el nombre del juego.

### playtime_forever
El número total de minutos jugados "registrados"
- 	Top 5 usuarios con más minutos jugados en total
-	Top 5 juegos con más minutos jugados
-	Promedio de minutos jugados por usuario
-	Promedio de minutos por juego

**Hallazgos**
* Los 10 mejores juegos representan aproximadamente el 41% del tiempo total de juego.
* El tiempo de juego promedio por usuario es de 1497 minutos
* El 50% de todos los usuarios juegan más de 887 minutos.
* El promedio de minutos por juego es de 277 minutos
* El 50% de los juegos se juegan durante más de 90 minutos en promedio

### playtime_2weeks
El número total de minutos jugados "registrados" en las últimas 2 semanas
- 	Top 5 usuarios con más minutos jugados en total
-	Top 5 juegos con más minutos jugados
-	Promedio de minutos jugados por usuario
-	Promedio de minutos por juego

# 3. Feature Engineering

## 3.1 Sentiment Analysis
To gain some insight and true value from our user's reviews we will conduct a Sentiment Analysis from the reviews of our users.
This **Sentiment Analysis** will then be used in our recommendation system, model.<br>
For this, we chose to work with NLTK pyhton libraries <br>
There is not a unique approach to Sentiment Analysis, all methods can deliver a good result. 
I our project we chose NLTK library and not TextBlob that is a much simpler approach:

NLTK (Natural Language Toolkit):

**Advantages**:
-   Provides a wide range of tools and resources for natural language processing tasks.
-   Flexible and customizable for advanced NLP tasks.
-   Well-established community and extensive documentation.

**Disadvantages**:
-   Requires more code and effort for simple tasks compared to TextBlob.
-   May be slower for certain tasks due to its comprehensive nature.

In our case we studied and implemented both methods, you can review in detail in [Feature Engineering](03.%20Feature%20Engineering.ipynb).

We proceeded following the process of:

1. **Data Preprocessing and Tokenization**: This step involves cleaning and preparing the text data for analysis. It includes tasks such as removing punctuation, converting text to lowercase, removing stop words, and handling special characters.
Tokenization involves breaking down the text into individual words or tokens.
> We chose to perform this by creating a new column, named 'clean_review' and passing a function to the 'review_text' column  content and storing this transformation, for each element in the 'clean_review' 

2. **Sentiment Analysis**: Once the text data is prepared, sentiment analysis can be performed. There are several libraries and techniques available for sentiment analysis, including lexicon-based approaches, machine learning models, and deep learning models.
We set to define the sentiment classification into:
*   Negative = 0 for all values where compound_score was under - 0.05
*   Neutral = 1 all other results nor negative nor positive and none values or emojis.
*   Positive = 2 for all values where compound_score was above 0.05
> For our work, we chose to work with Vader. ALthough Textblob is a suitable solution because is much faster and simpler, when comparing both results, the only disadvanage of NLTK was the use of resources, the model runs slow, but the outcome is better.
>NLTK is more assertive in classifing negative and positive sentiments, but even better for neurtral classification.
In the [Feature Engineering](03.%20Feature%20Engineering.ipynb) you can review in detail the process for both approaches and our analysis, and then in the [EDA_reviews](EDA_reviews.ipynb) you can find the comparison of the results for both approaches and more exploration.<br>

3. **Analysis, Visualization and Summary**: Finally, the results of the sentiment analysis can be analyzed and visualized to gain insights from the data.

* Resources:<br>
[Sentiment Analysis](https://www.youtube.com/watch?v=O_B7XLfx0ic)

[NLTK Sentiment Analysis](https://www.youtube.com/watch?v=XFoehWRzG-I)

[Sentiment_Analysis_article](https://www.kdnuggets.com/sentiment-analysis-in-python-going-beyond-bag-of-words)

## 3.2  Rating our users reviews in the reviews dataframe.
## Reviews Transformation
### We need to transform our users reviews, recommend button result and the rating from helpful (where other users qualified the review (rate) into a rating) in order to feed our recommendation system.<br>
For this we create a function that will consider all three items, when existing and average a rating that expresses the user's review in numbers.<br>
Overall, this function assigns a rating score based on the provided values for 'rate', 'sentiment', and 'recommend', considering different combinations of these values and following specific rules.<br>
*   If the 'rate' column is greater than 0:
        -If the 'recommend' column is 0:
            -If the 'sentiment' column is 0, the rating score is set to 0.
            -If the 'sentiment' column is 1, the rating score is calculated as 50 times the 'rate'.
            -If the 'sentiment' column is 2, the rating score is calculated as 75 times the 'rate'.
        -If the 'recommend' column is 1:
            -If the 'sentiment' column is 0, the rating score is calculated as 50 times the 'rate'.
            -If the 'sentiment' column is 1, the rating score is calculated as 75 times the 'rate'.
            -If the 'sentiment' column is 2, the rating score is calculated as 100 times the 'rate'.
*   If the 'rate' column is 0:
    - If the 'recommend' column is 0:
        - If the 'sentiment' column is 0, the rating score is set to 0.
        - If the 'sentiment' column is 1, the rating score is set to 50.
        - If the 'sentiment' column is 2, the rating score is set to 75.
    - If the 'recommend' column is 1:
        - If the 'sentiment' column is 0, the rating score is set to 50.
        - If the 'sentiment' column is 1, the rating score is set to 75.
        - If the 'sentiment' column is 2, the rating score is set to 100
Then we normalize this rating into a 1 to 5 score.<br>

This rating will be the engine for our Collaborative Filtering Recommender System and the Machine Learning KNN Recommender <br>

## 3.3 Collaborative Filtering Recommender

## 3.4 Machine Learning KNN Recommender

## 3.5 API Functions 
In this section of our notebook we are going to experiment and work on our requested API Functions.<br>
These are:<br>
1. **Developer** number of games and percentage of Free content by developer by year.<br>
2. **User_data**: how much money has the user spent, what percentage, from the total number of games the user owns, has the user recommended from the reviews.recommend and how many games he purchased.<br>
3. **User_for_genres**: this function must return the user with the largest amount of minutes for the given genres and a list of minutes accumulated per year since the release date<br>
4. **Best_developer_year**: this function returns the top 3 developers based on the largest amount of recommendations for the given year (reviews.recommend = True and sentiment = 2)<br>
5. **Developer_Reviews_Analyis**: given a developer, the function returns a dicctionary with developer as keys and the amount of Positive and Negative Sentiment Reviews.<br> 

In the notebook [Feature Engineering](03.%20Feature%20Engineering.ipynb) is the complete step by step to create each of this functions, and their input dataframes.

[Deploy an API in Render](https://medium.com/@iamgreatdiro/deploying-apis-on-render-a-step-by-step-guide-4ebe6a3fd377)